# 카글 텍스트 분류 - 합성곱 신경망 활용 접근방법

In [1]:
import sys
import os
import numpy as np
import json

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [2]:
DEFAULT_PATH ='~/.kaggle/competitions/word2vec-nlp-tutorial/'
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
INPUT_TRAIN_DATA_FILE_NAME = 'train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME = 'train_label.npy'
INPUT_TEST_DATA_FILE_NAME = 'test_input.npy'

DATA_CONFIGS_FILE_NAME = 'data_configs.json'

train_input_data = np.load(open(FILE_DIR_PATH + INPUT_TRAIN_DATA_FILE_NAME, 'rb'))
train_label_data = np.load(open(FILE_DIR_PATH + LABEL_TRAIN_DATA_FILE_NAME, 'rb'))
test_input_data = np.load(open(FILE_DIR_PATH + INPUT_TEST_DATA_FILE_NAME, 'rb'))

prepro_configs = None

with open(FILE_DIR_PATH + DATA_CONFIGS_FILE_NAME, 'r') as f:
    prepro_configs = json.load(f)

In [3]:
# 파라메터 변수
RNG_SEED = 1234
BATCH_SIZE = 128
NUM_EPOCHS = 10000
VOCAB_SIZE = len(prepro_configs)
EMB_SIZE = 128
VALID_SPLIT = 0.2
MAX_SEQ_LEN = 604 # 문장 최대 길이

input_train, input_eval, label_train, label_eval = train_test_split(train_input_data, train_label_data, test_size=VALID_SPLIT, random_state=RNG_SEED)

## tf.data 세팅

In [4]:
def mapping_fn(X, Y=None):
    input, label = {'x': X}, Y
    return input, label

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_train, label_train))
    dataset = dataset.shuffle(buffer_size=len(input_train))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)
    dataset = dataset.repeat(count=NUM_EPOCHS)

    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((input_eval, label_eval))
    dataset = dataset.shuffle(buffer_size=len(input_eval))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(mapping_fn)

    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

## 모델 세팅

In [12]:
def model_fn(features, labels, mode, params):

    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT
    
    #embedding layer를 선언합니다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['x'],
                    VOCAB_SIZE,
                    EMB_SIZE,
                    initializer=params['embedding_initializer']
                    )
    
    # 현재 모델이 학습모드인지 여부를 확인하는 변수입니다.
    training = (mode == tf.estimator.ModeKeys.TRAIN)    
    # embedding layer에 대한 output에 대해 dropout을 취합니다.
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.5,
                                   training=training)
    
    
    conv = tf.layers.conv1d(
            inputs=dropout_emb,
            filters=32,
            kernel_size=3,
            padding='same',
            activation=tf.nn.relu)
    
    pool = tf.reduce_max(input_tensor=conv, axis=1)
#     hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
#     dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
#     logits = tf.layers.dense(inputs=dropout_hidden, units=1, name='logits')
    hidden = layers.Dense(250, activation=tf.nn.relu)(pool)
    dropout_hidden = layers.Dropout(rate=0.2)(hidden)
    logits = layers.Dense(1, name='logits')(dropout_hidden)
    
    #prediction 진행 시, None
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        train_op = tf.train.AdamOptimizer(0.001).minimize(loss, global_step)

        return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
    
    elif EVAL:
        loss = tf.losses.sigmoid_cross_entropy(labels, logits)
        pred = tf.nn.sigmoid(logits)
        accuracy = tf.metrics.accuracy(labels, tf.round(pred))
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
        
    elif PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                'prob': tf.nn.sigmoid(logits)
            }
        )

In [13]:
params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}

model_dir = os.path.join(os.getcwd(), "data_out/checkpoint/cnn/")
os.makedirs(model_dir, exist_ok=True)

config_tf = tf.estimator.RunConfig()
config_tf._save_checkpoints_secs = 100
config_tf._keep_checkpoint_max =  2
config_tf._log_step_count_steps = 100

cnn_est = tf.estimator.Estimator(model_fn, model_dir=model_dir, config=config_tf, params=params)
cnn_est.train(train_input_fn) #학습하기

INFO:tensorflow:Using config: {'_tf_random_seed': None, '_service': None, '_save_checkpoints_steps': None, '_master': '', '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_task_id': 0, '_session_config': None, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_model_dir': '/home/companyai8way/ryan/classifier/data_out/checkpoint/cnn/', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9ae803bc88>, '_save_checkpoints_secs': 100, '_keep_checkpoint_max': 2, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_evaluation_master': '', '_device_fn': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/companyai8way/ryan/classifier/data_out/checkpoint/cnn/model.ckpt-17079
INFO:tensorflow:Running local_init_op.
IN

INFO:tensorflow:global_step/sec: 55.3315
INFO:tensorflow:loss = 0.00048557937, step = 23879 (1.807 sec)
INFO:tensorflow:global_step/sec: 58.5257
INFO:tensorflow:loss = 0.0017897304, step = 23979 (1.710 sec)
INFO:tensorflow:global_step/sec: 55.9566
INFO:tensorflow:loss = 0.0011317506, step = 24079 (1.785 sec)
INFO:tensorflow:global_step/sec: 55.7348
INFO:tensorflow:loss = 4.3058873e-05, step = 24179 (1.797 sec)
INFO:tensorflow:global_step/sec: 58.594
INFO:tensorflow:loss = 0.0002442683, step = 24279 (1.705 sec)
INFO:tensorflow:global_step/sec: 56.7205
INFO:tensorflow:loss = 0.0031995876, step = 24379 (1.762 sec)
INFO:tensorflow:global_step/sec: 56.8254
INFO:tensorflow:loss = 0.0017085858, step = 24479 (1.759 sec)


KeyboardInterrupt: 

In [14]:
cnn_est.evaluate(eval_input_fn) #평가하기

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-27-13:15:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/companyai8way/ryan/classifier/data_out/checkpoint/cnn/model.ckpt-22749
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-27-13:15:20
INFO:tensorflow:Saving dict for global step 22749: acc = 0.8762, global_step = 22749, loss = 0.72512066
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22749: /home/companyai8way/ryan/classifier/data_out/checkpoint/cnn/model.ckpt-22749


{'acc': 0.8762, 'global_step': 22749, 'loss': 0.72512066}

# 테스트 데이터셋

In [11]:
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.
test_input_data = np.load(open(FILE_DIR_PATH + INPUT_TEST_DATA_FILE_NAME, 'rb')) #테스트데이터 로드

predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":test_input_data}, shuffle=False) #numpy 형태로 저장
cnn_est.predict(input_fn=predict_input_fn)

predictions = np.array([p['prob'][0] for p in cnn_est.predict(input_fn=predict_input_fn)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/companyai8way/ryan/classifier/data_out/checkpoint/cnn/model.ckpt-17079
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [12]:
import pandas as pd
#테스트 데이터 로드
test = pd.read_csv(DEFAULT_PATH+"testData.tsv", header=0, delimiter="\t", quoting=3 )

print ("test dataset shape: {}".format(test.shape))

#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":list(predictions)} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv("./data_out/Bag_of_Words_model_test.csv", index=False, quoting=3 )

FileNotFoundError: File b'/home/companyai8way/.kaggle/competitions/word2vec-nlp-tutorial/testData.tsv' does not exist